# Aim is to get zip code for each station for each day

In [1]:
import pandas as pd
from geopy.distance import vincenty
import numpy as np
import densitymetric
import matplotlib.mlab as mlab

In [2]:
# skip first 4 cells and just read in the csv files
population = pd.read_csv('../Data/Boston/population.csv')
employee = pd.read_csv('../Data/Boston/employee.csv')
station = pd.read_csv('../Data/Boston/hubway_stations.csv')

# Generate a regular grid to interpolate the data.
xmin = 42.29
xmax = 42.43
ymin = -71.19
ymax = -70.98
nx = 200
ny = 200
xvec = np.linspace(xmin, xmax, nx)
yvec = np.linspace(ymin, ymax, ny)
xi, yi = np.meshgrid(xvec, yvec)
x = employee['latitude']
y = employee['longitude']
z = employee['EMP']
workmap = mlab.griddata(x, y, z, xi, yi, interp='linear')
x = population['latitude']
y = population['longitude']
z = population['SUBHD0401']
popmap = mlab.griddata(x, y, z, xi, yi, interp='linear')
#origdest = origin * destination

In [3]:
stationlat = station['lat'].values
stationlong = station['lng'].values
closeradius = 0.5
scaledistance = 1.5
scores = densitymetric.getscores(xvec, yvec, popmap, workmap, stationlat, stationlong, closeradius, scaledistance)

> /Users/rbussman/Proposals/Resume/Insight/bikeshare/Code/densitymetric.py(58)coupling()
-> latmesh, longmesh = np.meshgrid(latvec, longvec)
(Pdb) n
> /Users/rbussman/Proposals/Resume/Insight/bikeshare/Code/densitymetric.py(59)coupling()
-> stationlatmesh, stationlongmesh = np.meshgrid(stationlat, stationlong)
(Pdb) n
> /Users/rbussman/Proposals/Resume/Insight/bikeshare/Code/densitymetric.py(60)coupling()
-> stationdistance = griddata((latmesh, longmesh), distancemap,
(Pdb) n
> /Users/rbussman/Proposals/Resume/Insight/bikeshare/Code/densitymetric.py(61)coupling()
-> (stationlatmesh, stationlongmesh))
(Pdb) n
ValueError: ValueErr...points',)
> /Users/rbussman/Proposals/Resume/Insight/bikeshare/Code/densitymetric.py(61)coupling()
-> (stationlatmesh, stationlongmesh))
(Pdb) exit


BdbQuit: 

In [23]:
radius = 0.5
popdensity = []
empdensity = []
for i in range(len(station)):
    inlat = station['lat'][i]
    inlong = station['lng'][i]
    dmetric = densitymetric.getval(xvec, yvec, popmap, workmap, inlat, inlong, radius)
    print(i, dmetric)
    popdensity.append(dmetric[0])
    empdensity.append(dmetric[1])

station['popdensity'] = popdensity
station['empdensity'] = empdensity

(20099.358399239834, 16692.753933757544)
(0, (20099.358399239834, 16692.753933757544))
(18086.238604933023, 21454.659662862308)
(1, (18086.238604933023, 21454.659662862308))
(19797.945738072274, 21044.219877649099)
(2, (19797.945738072274, 21044.219877649099))
(18666.673243828118, 13881.169877648354)
(3, (18666.673243828118, 13881.169877648354))
(16503.835555016994, 9005.8250046214089)
(4, (16503.835555016994, 9005.8250046214089))
(16280.786424988881, 9477.040112003684)
(5, (16280.786424988881, 9477.040112003684))
(20584.230737170205, 12054.449577254825)
(6, (20584.230737170205, 12054.449577254825))
(21684.013391648419, 16397.786388501525)
(7, (21684.013391648419, 16397.786388501525))
(19313.55585192889, 13586.902291264385)
(8, (19313.55585192889, 13586.902291264385))
(17855.964344034437, 19837.757629573345)
(9, (17855.964344034437, 19837.757629573345))
(16396.310222879052, 20282.492387592793)
(10, (16396.310222879052, 20282.492387592793))
(19240.88901425153, 14968.563885415206)
(11, (

In [7]:
station = station.rename(columns = {'id': 'stationid'})

In [8]:
station

,stationid,terminal,station,municipal,lat,lng,status,popdensity,empdensity
0,3,B32006,Colleges of the Fenway,Boston,42.340021,-71.100812,Existing,20099.358399,16692.75
1,4,C32000,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616,Existing,18086.238605,21454.66
2,5,B32012,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179,Existing,19797.945738,21044.22
3,6,D32000,Cambridge St. at Joy St.,Boston,42.361285,-71.065140,Existing,18666.673244,13881.17
4,7,A32000,Fan Pier,Boston,42.353412,-71.044624,Existing,16503.835555,9005.825
5,8,A32001,Union Square - Brighton Ave. at Cambridge St.,Boston,42.353334,-71.137313,Existing,16280.786425,9477.04
6,9,A32002,Agganis Arena - 925 Comm Ave.,Boston,42.351313,-71.116174,Existing,20584.230737,12054.45
7,10,A32003,B.U. Central - 725 Comm. Ave.,Boston,42.350075,-71.105884,Existing,21684.013392,16397.79
8,11,A32004,Longwood Ave / Binney St,Boston,42.338629,-71.106500,Existing,19313.555852,13586.9
9,12,B32002,Ruggles Station / Columbus Ave.,Boston,42.335911,-71.088496,Existing,17855.964344,19837.76


In [9]:
station.to_csv('../Data/Boston/stationdensity.csv')

In [10]:
zipcode = pd.read_csv('../Data/zipcode/zipcode.csv', converters={'zip': lambda x: str(x)})

In [ ]:

loc1 = (41.49008, -71.312796)
loc2 = (41.499498, -81.695391)
print(vincenty(newport_ri, cleveland_oh).miles)
538.3904451566326

In [4]:
points = np.random.rand(1000, 2)

In [5]:
points.shape

(1000, 2)